# Notebook for data reduction of PSW

In [1]:
# dependent packages
import numpy as np
import xarray as xr
import fmflow as fm
from astropy.modeling.models import Linear1D
from astropy.modeling.fitting import LinearLSQFitter

from bokeh.plotting import figure, show, save, output_notebook
output_notebook()

Loading BokehJS ...

## Config

In [2]:
path_cal = 'xffts20181005143750.xfftsx.02.nc'
path_sci = 'xffts20181005143813.xfftsx.02.nc'

Tamb = 273.0 # K
freq = 157.2-0.1 + np.linspace(0.0, 2.5, 2**15)

## Step 1: Create FM array (ON, OFF, and R)

In [3]:
with xr.open_dataset(path_sci) as ds:
    array     = ds['array'].copy().values
    integtime = ds['integtime'].copy().values
    scantype  = ds['bufpos'].copy().values
    scanno    = ds['scancount'].copy().values

tcoords  = {'scanno': scanno}
chcoords = {'fsig': freq}

P = fm.array(array/integtime[:,None],
             tcoords=tcoords, chcoords=chcoords)

Pon  = P[scantype == 'ON']
Poff = P[scantype == 'REF']

In [4]:
with xr.open_dataset(path_cal) as ds:
    array     = ds['array'].copy().values
    integtime = ds['integtime'].copy().values

Pr = fm.array(array/integtime[:,None]).mean('t')

## Step 2: Chopper-wheel calibration

In [5]:
Tcal = fm.zeros_like(Pon)

for no in np.unique(Pon.scanno):
    # ON array of single scan
    Pon_ = Pon[Pon.scanno==no]

    # OFF array for single scan
    Poff_l = Poff[Poff.scanno==no-1].mean('t')
    Poff_r = Poff[Poff.scanno==no+1].mean('t')
    Poff_ = (Poff_l+Poff_r) / 2
    
    # calibrated array
    Tcal_ = Tamb * (Pon_-Poff_) / (Pr-Poff_)
    Tcal[Tcal.scanno==no] = Tcal_

## Step 3: Baseline subtraction

In [6]:
fitter = LinearLSQFitter()
linear = Linear1D(0, 0)

for spec in Tcal.values:
    model = fitter(linear, freq, spec)
    spec -= model(freq)

## Step 4: Channel binning

In [7]:
Tcal_bin = fm.chbinning(Tcal, 32)

## Step 5: Plot spectrum

In [8]:
p = figure(plot_width=1200, plot_height=400)
p.line(Tcal_bin.fsig.values[10:-10], Tcal_bin.mean('t').values[10:-10])
show(p)